# Transfer free energy benchmarking

This notebook will extract the aqueous and non-aqueous solvation free energies computed for a force field using absolv and compare them with experimental values. We will also calculate transfer free energies.

Assuming the absolv calculations have been run and the results are stored in folders `aqueous-results` & `non-aqueous-results`. 

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Dict, Tuple, Union
import numpy as np
from openff.evaluator.datasets import PhysicalPropertyDataSet
from openff.units import unit
from absolv.models import TransferFreeEnergyResult
import os
from openmm import unit
import seaborn as sns
import scipy
sns.set_theme()

Start by extracting the experimental values for the aqueous and non-aqueous solvation free energies, note that the MNsol values are not included here and should be generated first.

In [ ]:
experiment_hydration = {}
experiment_non_aqueous = {}
fsolv = PhysicalPropertyDataSet.from_json("../../../data-set-curation/physical-property/physical-data-sets/fsolv-filtered.json")
mnsol = PhysicalPropertyDataSet.from_json("../../../data-set-curation/physical-property/physical-data-sets/mnsol-filtered.json")

def extract_dataset(dataset: PhysicalPropertyDataSet, solute_only: bool) -> Dict[Union[str, Tuple[str, str]], Tuple[float, float]]:
    """Extract the experiemtnal property data into a dictionary of solute or solute solvent mixures with the experimental value and error."""
    dataset_data = {}
    for property in dataset.properties:
        components = property.substance.components
        solute = [component.smiles for component in components if component.role.name == "Solute"][0]
        index = solute
        if not solute_only:
            solvent = [component.smiles for component in components if component.role.name == "Solvent"][0]
            index = (solute, solvent)

        dataset_data[index] = (property.value.m / 4.184, property.uncertainty.m / 4.184 )
    return dataset_data

experiment_hydration = extract_dataset(dataset=fsolv, solute_only=True)
experiment_non_aqueous = extract_dataset(dataset=mnsol, solute_only=False)


Gather the DE-FF calculated values for these two datasets

In [ ]:
dexp_hyd_results, dexp_non_aqu_results = [], []
for result_file in os.listdir("aqueous-results"):
    if "schema" not in result_file:
        continue
    dexp_hyd_results.append(TransferFreeEnergyResult.parse_file(os.path.join("aqueous-results", result_file)))

for result_file in os.listdir("non-aqueous-results"):
    if "schema" not in result_file:
        continue
    dexp_non_aqu_results.append(TransferFreeEnergyResult.parse_file(os.path.join("non-aqueous-results", result_file)))
    
def gather_absolv_results(results: List[TransferFreeEnergyResult], solute_only: bool) -> Dict[Union[str, Tuple[str, str]], Tuple[float, float]]:
    """Extract the absolv results from a list of results schemas and store into a dictionary where the key is the solute or solute solvent mixture."""
    dataset_results = {}
    for result in results:
        solute = list(result.input_schema.system.solutes.keys())[0]
        index = solute
        if not solute_only:
            solvent = list(result.input_schema.system.solvent_b.keys())[0]
            index = (solute, solvent)

        calc_value, error = result.delta_g_from_a_to_b_with_units
        dataset_results[index] = (calc_value.value_in_unit(unit.kilocalorie_per_mole), error.value_in_unit(unit.kilocalorie_per_mole))
    return dataset_results

dexp_hyd_values = gather_absolv_results(results=dexp_hyd_results, solute_only=True)
dexp_non_aqu_values = gather_absolv_results(results=dexp_non_aqu_results, solute_only=False)

Collect together the aqueous non-aqueous and transfer free energies and experimental values in the same ordering for comparison.

In [ ]:
experiment_trans, dexp_trans = [], []
experiment_hyd, dexp_hyd = [], []
experiment_non_aqu, dexp_non_aqu = [], []
for mixture, non_aqu_data in dexp_non_aqu_values.items():
    solute, solvent = mixture
    experiment_non_aqu.append(experiment_non_aqueous[mixture][0])
    dexp_non_aqu.append(non_aqu_data[0])
    if solute in dexp_hyd_values:
        # calculate the transfer free energy
        experiment_trans.append(experiment_non_aqueous[mixture][0] - experiment_hydration[solute][0])
        dexp_trans.append(non_aqu_data[0] - dexp_hyd_values[solute][0])

# store the hydration free energies
for solute, hydration_data in dexp_hyd_values.items():
    try:
        experiment_hyd.append(experiment_hydration[solute][0])
        dexp_hyd.append(hydration_data[0])
    except KeyError:
        continue


Define analysis funtions to calculate reported errors and general bootstrapping method

In [ ]:
def reg_line(x, slope_value, intercept_value):
    "Calculate the regression line used in scatter plots."
    return intercept_value + x * slope_value

In [ ]:
def MUE(experiment_values: np.array, calculated_values: np.array) -> float:
    "Calculate the mean unsigned error between a set experimental and calculated values."
    return np.mean(np.absolute(experiment_values - calculated_values))

def RMSE(experiment_values: np.array, calculated_values: np.array) -> float:
    "Calculate the root mean squared error between a set experimental and calculated values."
    return np.sqrt(np.mean((experiment_values - calculated_values) ** 2))

def CORRELATION(experiment_values: np.array, calculated_values: np.array) -> float:
    "Calculate the r squared correlation between a set experimental and calculated values."
    _, _, r_value, _, _ = scipy.stats.linregress(experiment_values, calculated_values)
    return r_value**2

def KTAU(experiment_values: np.array, calculated_values: np.array) -> float:
    "Calculate the kendal tau between a set experimental and calculated values."
    return scipy.stats.kendalltau(experiment_values, calculated_values)[0]

def MSE(experiment_values: np.array, calculated_values: np.array) -> float:
    "Calculate the mean signed error between a set experimental and calculated values."
    return np.mean(calculated_values - experiment_values)

In [ ]:
def compute_bootstraped_statistics(refernce_values: List[float], calculated_values: List[float], bootstrap_iterations: int = 1000, interval: float = 0.95) -> Dict[str, float]:
    """
    Compute bootstraped statistics for each of the defined error metrics.
    """
    # make a list of experimental and predicted values
    experimental_values = np.array(refernce_values)
    prediction_values = np.array(calculated_values)
    mue = MUE(experiment_values=experimental_values, calculated_values=prediction_values)
    rmse = RMSE(experimental_values, prediction_values)
    correlation = CORRELATION(experimental_values, prediction_values)
    tau = KTAU(experimental_values, prediction_values)
    mse = MSE(experimental_values, prediction_values)
    samples = len(experimental_values)
    sample_statistics = defaultdict(list)
    for _ in range(bootstrap_iterations):
        sample_indicies = np.random.randint(low=0, high=samples, size=samples)
        sample_experiment_values = experimental_values[sample_indicies]
        sample_prediction_values = prediction_values[sample_indicies]

        sample_statistics["MUE"].append(MUE(experiment_values=sample_experiment_values, calculated_values=sample_prediction_values))
        sample_statistics["RMSE"].append(RMSE(experiment_values=sample_experiment_values, calculated_values=sample_prediction_values))
        sample_statistics["R^2"].append(CORRELATION(experiment_values=sample_experiment_values, calculated_values=sample_prediction_values))
        sample_statistics["KTAU"].append(KTAU(experiment_values=sample_experiment_values, calculated_values=sample_prediction_values))
        sample_statistics["MSE"].append(MSE(experiment_values=sample_experiment_values, calculated_values=sample_prediction_values))
        
     # Compute the confidence intervals.
    lower_percentile_index = int(bootstrap_iterations * (1 - interval) / 2)
    upper_percentile_index = int(bootstrap_iterations * (1 + interval) / 2)

    final_stats = {}
    for metric, base_value in [("MUE", mue), ("RMSE", rmse), ("R^2", correlation), ("KTAU", tau), ("MSE", mse)]:
        final_stats[metric] = base_value
        sorted_samples = np.sort(sample_statistics[metric])
        final_stats[metric + "_lower"] = sorted_samples[lower_percentile_index]
        final_stats[metric + "_higher"] = sorted_samples[upper_percentile_index]

    return final_stats

In [ ]:
# calculate the correlation and error
dexp_slope, dexp_intercept, dexp_r_value, _, _, = scipy.stats.linregress(experiment_trans, dexp_trans)
dexp_stats = compute_bootstraped_statistics(experiment_trans, dexp_trans)
dexp_stats

In [ ]:
plt.scatter(experiment_trans, dexp_trans, label=f"""RMSE: {round(dexp_stats['RMSE'], 3)} kcal/mol
MUE: {round(dexp_stats['MUE'], 3)} kcal/mol""", alpha=0.5)
x = np.linspace(-10, 5, 1000)
dexp_regression_line = reg_line(x, slope_value=dexp_slope, intercept_value=dexp_intercept)
plt.plot(x, x, "k--")
plt.plot(x, dexp_regression_line, label=rf"$r^{2}$={round(dexp_stats['R^2'], 3)}", color="red")
# plt.plot(x, sage_regression_line)
plt.ylim(-9.5, 5)
plt.xlim(-9.5, 5)
fontdict={"size": 18}
plt.xlabel(r"Experimental $\Delta$G$_{trans}$ (kcal/mol)", fontdict=fontdict)
plt.ylabel(r"Calculated $\Delta$G$_{trans}$ (kcal/mol)", fontdict=fontdict)
plt.legend()
plt.tight_layout()
plt.show()


Now repeat for the other properties.

In [ ]:
# calculate the correlation
dexp_slope, dexp_intercept, dexp_r_value, _, _ = scipy.stats.linregress(experiment_hyd, dexp_hyd)
dexp_hyd_stats = compute_bootstraped_statistics(experiment_hyd, dexp_hyd)
dexp_hyd_stats

In [ ]:
plt.scatter(experiment_hyd, dexp_hyd, label=rf"DEXP $r^{2}$={round(dexp_hyd_stats['R^2'], 3)}", alpha=0.5)
x = np.linspace(-12, 5, 1000)
dexp_regression_line = reg_line(x, slope_value=dexp_slope, intercept_value=dexp_intercept)
plt.plot(x, dexp_regression_line)
plt.plot(x, x, "k--")
plt.ylim(-12, 5)
plt.xlim(-12, 5)
fontdict={"size": 18}
plt.xlabel(r"Experimental $\Delta$G$_{hyd}$ (kcal/mol)", fontdict=fontdict)
plt.ylabel(r"Calculated $\Delta$G$_{hyd}$ (kcal/mol)", fontdict=fontdict)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# calculate the correlation
dexp_slope, dexp_intercept, dexp_r_value, _, _ = scipy.stats.linregress(experiment_non_aqu, dexp_non_aqu)
dexp_non_aqu_stats = compute_bootstraped_statistics(experiment_non_aqu, dexp_non_aqu)
dexp_non_aqu_stats

In [ ]:
plt.scatter(experiment_non_aqu, dexp_non_aqu, label=rf"DEXP $r^{2}$={round(dexp_non_aqu_stats['R^2'], 3)}", alpha=0.5)
x = np.linspace(-12, 1, 1000)
dexp_regression_line = reg_line(x, slope_value=dexp_slope, intercept_value=dexp_intercept)
plt.plot(x, x, "k--")
plt.plot(x, dexp_regression_line)
plt.ylim(-12, 0)
plt.xlim(-12, 0)
fontdict={"size": 18}
plt.xlabel(r"Experimental $\Delta$G$_{sol}$ (kcal/mol)", fontdict=fontdict)
plt.ylabel(r"Calculated $\Delta$G$_{sol}$ (kcal/mol)", fontdict=fontdict)
plt.legend()
plt.tight_layout()
plt.show()